In [ ]:
from notebook_prelude import *

In [ ]:
GRAPHS = {}
cmaps = dataset_helper.get_all_cached_graph_datasets(graph_type=TYPE_CONCEPT_MAP)
for cache_file in cmaps:
    dataset = filename_utils.get_dataset_from_filename(cache_file)
    X, Y = dataset_helper.get_dataset_cached(cache_file=cache_file)
    X_g = graph_helper.get_graphs_only(X)
    GRAPHS[dataset] = (X_g, Y)


In [ ]:
cmap = dataset_helper.get_all_cached_graph_datasets(dataset_name='ted_talks', graph_type=TYPE_CONCEPT_MAP)[0]
X_, Y = dataset_helper.get_dataset_cached(cmap)

In [ ]:
figsize=(13, 13)

X = graph_helper.get_graphs_only(X_)
candidates = [idx for idx, g in enumerate(X) if len(g.nodes()) < 15 and len(g.nodes()) > 5]
idx = np.random.choice(candidates)
g = X[idx]
fig, ax = plt.subplots(figsize=figsize)
pos = nx.layout.circular_layout(G=g)#, iterations=100)
nx.draw_networkx(g, node_size=4, pos=pos, ax = ax)
edges = {(s, t): data['name'] for s, t, data in g.edges(data=True)}
nx.draw_networkx_edge_labels(g, pos=pos, edge_labels=edges)
cleanup_axes(ax)

In [ ]:
import experiments
from experiments import experiment_helper
experiment_helper.save_experiment_params_as_experiment_config()
print('\n'.join([filename_utils.get_dataset_from_filename(x) for x in dataset_helper.get_all_cached_graph_datasets(graph_type=TYPE_CONCEPT_MAP)]))


In [ ]:
X, Y = dataset_helper.get_dataset('imsdb')
word_counts = [len(x.split()) for x in X]

In [ ]:
pd.DataFrame(word_counts, columns = ['word_counts']).plot(kind='hist', bins=120)
len(word_counts)

In [ ]:
FIG_SIZE = (30, 30)
#FIG_SIZE = (5, 5)
g = nx.read_gml('tmp/concept-graph.graph')

node_mapping = {}
for node, data in g.nodes(data=True):
    node_mapping[node] = data['name']
node_labels = list(node_mapping.values())
assert len(node_labels) == len(set(node_labels))


nx.relabel_nodes(g, node_mapping, copy=False)
edges = g.edges(data=True)
edge_labels = {(k, v): data['name'] for k, v, data in edges}

for layout in [nx.layout.circular_layout, nx.layout.fruchterman_reingold_layout, nx.layout.random_layout, nx.layout.shell_layout, nx.layout.spectral_layout, nx.layout.spring_layout]:
    fig, ax = plt.subplots(figsize=FIG_SIZE)
    pos = layout(g)
    nx.draw_networkx(g, ax=ax, pos=pos, node_size=10)
    nx.draw_networkx_edge_labels(g, pos, edge_labels=edge_labels)
    ax.grid(False)
    ax.set_title(layout.__name__)
    fig.tight_layout()
    save_fig(fig, 'concept_map_{}'.format(layout.__name__), folder='tmp/concept_map_example')